##Important Notes:

1) Make sure GPU is enabled, go to edit->notebook settings->Hardware Accelerator GPU

2) Make a copy to your google drive, click on copy to drive in panel

Note: Colab will reset after 12 hours make sure to save your model checkpoints to google drive around 10-11 hours mark or before, then go to runtime->reset all runtimes. Now copy your train model back into colab and start training again from the previous checkpoint. [More info](https://stackoverflow.com/questions/55050988/can-i-run-a-google-colab-free-edition-script-and-then-shutdown-my-computer)

Thanks [@ak9250](https://github.com/ak9250/gpt-2-colab/blob/master/GPT_2.ipynb) and [@nshepperd](https://github.com/nshepperd/gpt-2) for sharing their codes.

In [ ]:
!nvidia-smi

Wed Nov 20 11:27:05 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.50       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## 1. Setting Up

Clone and cd into repo, nshepperd's fork https://github.com/nshepperd/gpt-2

In [ ]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 310, done.
remote: Total 310 (delta 0), reused 0 (delta 0), pack-reused 310
Receiving objects: 100% (310/310), 4.40 MiB | 3.04 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [ ]:
cd gpt-2

/content/gpt-2


Install requirements

In [ ]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 2.7MB/s 
     |████████████████████████████████| 604kB 7.2MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=6ad555f2510dcfc82ec78c179193ceb46433fd8fab0ae4ea032854b1000b4078
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533175 sha256=cfa128bab2011fe0a79c1a70f0ef29170328bbc4b185500a0123f9539aac3e3e
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


Mount drive to access google drive for saving and accessing checkpoints later. Have to log in to your google account

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Download the model data

In [ ]:
!python3 download_model.py 117M

In [ ]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 822kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 62.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 845kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:17, 79.4Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 7.39Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 61.0Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 53.6Mit/s]                                                       


encoding

In [ ]:
!export PYTHONIOENCODING=UTF-8

Fetch checkpoints if you have them saved in google drive

In [ ]:
!cp -r /content/drive/My\ Drive/checkpoint/ /content/gpt-2/ 

## 2. Training
Let's get our train on! In this case the file is ```dataset.txt``` in your drive folder. To do this, just rename your text file as ```dataset.txt``` and copy to root of your google drive.


For small datasets(< 10 MB) you will have to be sure not to run the fine-tuning for too long or you will overfit badly. Roughly, expect interesting results within minutes to hours in the 1-10s of megabyte ballpark, and below this you may want to stop the run early as fine-tuning can be very fast.

Big datasets (>100 MB) might not fit the all GPUs of colab. If you get a memory error, either restart runtime until you get gpu with 16 GB memory(P100 or T40), or make your dataset smaller.

Copy ```dataset.txt``` from google drive

In [ ]:
!cp /content/drive/My\ Drive/dataset.txt dataset.txt


Start training, --model_name '345M' uses the 345 model. Default learning rate is 0.00002, it is good for starting fine tuning.

In [ ]:
!PYTHONPATH=src ./train.py --dataset dataset.txt --model_name '345M' --learning_rate 0.00002


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2019-11-20 09:26:32.317281: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-11-20 09:26:32.319539: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2d75480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-11-20 09:26:32.319587: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-11-20 09:26:32.325258: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2019-11-20 09:26:32.539441: I tensorflow/stream_executor/cu

## Save Checkpoints
Save our checkpoints to start training again later

In [ ]:
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/

## 3. Testing

Load your trained model for use in sampling below (117M or 345M)

In [ ]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [ ]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Generate conditional samples from the model given a prompt you provide -  change top-k hyperparameter if desired (default is 40),  if you're using 345M, add "--model-name 345M"

In [ ]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "345M"

To check flag descriptions, use:

In [ ]:
!python3 src/interactive_conditional_samples.py -- --help

Generate unconditional samples from the model,  if you're using 345M, add "--model-name 345M"

In [ ]:
!python3 src/generate_unconditional_samples.py --model_name "345M" --nsamples 3 | tee /tmp/samples

To check flag descriptions, use:

In [ ]:
!python3 src/generate_unconditional_samples.py -- --help

Check output similarity with the dataset.  [(source)](https://jmonlong.github.io/Hippocamplus/2018/04/16/text-similarity/)

In [ ]:
!wget https://raw.githubusercontent.com/jmonlong/Hippocamplus/config/python/simText/simText.py
!python2 simText.py -s 0.5 -1 dataset.txt -2 /tmp/samples 